# Библиотеки

In [1]:
import numpy as np
import pandas as pd
from PIL import Image

import keras
from keras.models import Model
from keras.layers import Input, Convolution2D, MaxPooling2D, Dense, Dropout, Flatten
from keras.utils import np_utils

import cv2
from tqdm import tqdm

Using TensorFlow backend.


# Загрузка и обработка данных

In [2]:
core_dir = 'C:\\Users\\horch\\Desktop\\Data mining\\Kaggle Amazon Rainforest\\'
train_dir = core_dir + 'train-jpg\\'
test_dir = core_dir + 'test-jpg\\'

cathegories = ['agriculture', 'artisinal_mine', 'bare_ground', 
                      'blooming', 'blow_down', 'clear', 'cloudy', 'conventional_mine', 
                      'cultivation', 'habitation', 'haze', 'partly_cloudy', 'primary', 
                      'road', 'selective_logging', 'slash_burn', 'water']

train_data = pd.read_csv(core_dir + 'train.csv')
train_data.head()

,image_name,tags
0,train_0,haze primary
1,train_1,agriculture clear primary water
2,train_2,clear primary
3,train_3,clear primary
4,train_4,agriculture clear habitation primary road


# Функции

подготовка обучающей выборки по имени категории
принимает категорию, возвращает массив из 0 и 1 (1 - файл содержит категорию, 0 - иначе)

In [3]:
def TrainingSetByCathegory(cathegory):
    df = pd.DataFrame()
    
    df['image_name'] = train_data.image_name.values
    df['indicator'] = np.zeros(train_data.shape[0])
    
    df.loc[df['image_name'].isin(cathegory_dict[cathegory]), ['indicator']] = 1
    
    return df.indicator.values

функция предназначенная для конвертации .jpg изображения в numpy массив
если transparency = False, то каждая точка представляется 3 числами иначе - 4

In [4]:
def ImageToNumpy(img_name, img_dir = train_dir, img_type = '.jpg', transparency = False):
    img = Image.open(train_dir + img_name + img_type)
    img.load()
    data = np.asarray( img, dtype="int32" )
    img.close()
    
    if transparency:
        return data
    else:
        return data[:, :, 0:3]

# Вычисления

создадим словарь категорий
каждой категории будет соответствовать список файлов, в которых данная категория присутствует

In [5]:
cathegory_dict = {cathegory: [] for cathegory in cathegories}

ind = 0
for tag in train_data.tags.values:
    file_name = train_data.image_name[ind]
    
    cathegory_names_list = tag.split(' ')
    
    for cathegory in cathegory_names_list:
        cathegory_dict[cathegory].append(file_name)
    
    ind += 1   

In [14]:
img = cv2.imread(train_dir + train_data.image_name[0] + '.jpg')
img.shape
res = cv2.resize(img, (32, 32))
res.shape

(32, 32, 3)

формирование обучающей выборки

In [6]:
X = []

for img_name in tqdm(train_data.image_name.values):
    
    img = cv2.imread(train_dir + img_name + '.jpg')
    res = cv2.resize(img, (32, 32))
    X.append(res)
    
X = np.array(X, np.float16) / 255.

100%|████████████████████████████████████| 40479/40479 [08:25<00:00, 80.09it/s]


In [8]:
X.shape

(40479, 32, 32, 3)

# Настройка и  обучение сети для одной категории

In [15]:
batch_size = 32 # in each iteration, we consider 32 training examples at once
num_epochs = 20 # we iterate 200 times over the entire training set
kernel_size = 3 # we will use 3x3 kernels throughout
pool_size = 2 # we will use 2x2 pooling throughout
conv_depth_1 = 32 # we will initially have 32 kernels per conv. layer...
conv_depth_2 = 64 # ...switching to 64 after the first pooling layer
drop_prob_1 = 0.25 # dropout after pooling with probability 0.25
drop_prob_2 = 0.5 # dropout in the FC layer with probability 0.5
hidden_size = 512 # the FC layer will have 512 neurons

In [13]:
# загрузка изображений и их приведение к подходящему для обработки виду
num_train = 40479
depth = 32 
height = 32
width = 3 
num_classes = len(cathegories)
y_train = TrainingSetByCathegory(cathegories[0]) #y_train labels (для категории_0 1, если подходит, 0-иначе)
Y_train = np_utils.to_categorical(y_train, num_classes) # One-hot encode the labels
X_train = X

In [16]:
inp = Input(shape=(depth, height, width)) # N.B. depth goes first in Keras!
# Conv [32] -> Conv [32] -> Pool (with dropout on the pooling layer)
conv_1 = Convolution2D(conv_depth_1, kernel_size, kernel_size, border_mode='same', activation='relu')(inp)
conv_2 = Convolution2D(conv_depth_1, kernel_size, kernel_size, border_mode='same', activation='relu')(conv_1)
pool_1 = MaxPooling2D(pool_size=(pool_size, pool_size))(conv_2)
drop_1 = Dropout(drop_prob_1)(pool_1)
# Conv [64] -> Conv [64] -> Pool (with dropout on the pooling layer)
conv_3 = Convolution2D(conv_depth_2, kernel_size, kernel_size, border_mode='same', activation='relu')(drop_1)
conv_4 = Convolution2D(conv_depth_2, kernel_size, kernel_size, border_mode='same', activation='relu')(conv_3)
pool_2 = MaxPooling2D(pool_size=(pool_size, pool_size))(conv_4)
drop_2 = Dropout(drop_prob_1)(pool_2)
# Now flatten to 1D, apply FC -> ReLU (with dropout) -> softmax
flat = Flatten()(drop_2)
hidden = Dense(hidden_size, activation='relu')(flat)
drop_3 = Dropout(drop_prob_2)(hidden)
out = Dense(num_classes, activation='softmax')(drop_3)

model = Model(input=inp, output=out) # To define a model, just specify its input and output layers

model.compile(loss='categorical_crossentropy', # using the cross-entropy loss function
              optimizer='adam', # using the Adam optimiser
              metrics=['accuracy']) # reporting the accuracy

model.fit(X_train, Y_train, # Train the model using the training set...
          batch_size=batch_size, nb_epoch=num_epochs,
          verbose=1, validation_split=0.1) # ...holding out 10% of the data for validation
model.evaluate(X_test, Y_test, verbose=1) # Evaluate the trained model on the test set!

D:\Users\horch\Anaconda2\envs\tensorflow\lib\site-packages\ipykernel\__main__.py:3: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), padding="same", activation="relu")`
  app.launch_new_instance()
D:\Users\horch\Anaconda2\envs\tensorflow\lib\site-packages\ipykernel\__main__.py:4: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), padding="same", activation="relu")`
D:\Users\horch\Anaconda2\envs\tensorflow\lib\site-packages\ipykernel\__main__.py:8: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), padding="same", activation="relu")`
D:\Users\horch\Anaconda2\envs\tensorflow\lib\site-packages\ipykernel\__main__.py:9: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), padding="same", activation="relu")`
D:\Users\horch\Anaconda2\envs\tensorflow\lib\site-packages\ipykernel\__main__.py:18: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outp

Train on 36431 samples, validate on 4048 samples
Epoch 1/20
36431/36431 [==============================] - 791s - loss: 0.5240 - acc: 0.7367 - val_loss: 0.4459 - val_acc: 0.7789
Epoch 2/20
36431/36431 [==============================] - 820s - loss: 0.4010 - acc: 0.8165 - val_loss: 0.4270 - val_acc: 0.8281
Epoch 3/20
36431/36431 [==============================] - 792s - loss: 0.3600 - acc: 0.8409 - val_loss: 0.3464 - val_acc: 0.8446
Epoch 4/20
36431/36431 [==============================] - 788s - loss: 0.3497 - acc: 0.8450 - val_loss: 0.3346 - val_acc: 0.8530
Epoch 5/20
36431/36431 [==============================] - 783s - loss: 0.3459 - acc: 0.8503 - val_loss: 0.3548 - val_acc: 0.8461
Epoch 6/20
36431/36431 [==============================] - 781s - loss: 0.3373 - acc: 0.8528 - val_loss: 0.3381 - val_acc: 0.8582
Epoch 7/20
36431/36431 [==============================] - 784s - loss: 0.3321 - acc: 0.8536 - val_loss: 0.3352 - val_acc: 0.8466
Epoch 8/20
36431/36431 [========================

NameError: name 'X_test' is not defined

In [17]:
# Генерируем описание модели в формате json
model_json = model.to_json()
# Записываем модель в файл
json_file = open("model.json", "w")
json_file.write(model_json)
json_file.close()

In [18]:
model.save_weights("model.h5")